# Slide puzzle (corridor) (****)

Slide coins to empty positions. The red coin should end up at the empty spot at the right
end. Model this with SMT and let Z3 find a solution.

![./slide_puzzle_corridor.png](./slide_puzzle_corridor.png)

## Approach

I did not want to use my code from previous semester since A) I lost my train of thought about that approach since I did not document it very well, and B) after some re-evaluation of the code I could not forsee whether the old approach would even work or not, so I decided to start from scratch.

Each time I wanted to test something, or could not get something to work, I would make a copy of my file and increment a counter by 1 in the filename, so that I could keep track of my progress this time. Each file is a step in the process of getting to the final solution. I will explain each file in the order they were created.

---

### Premise

The entire premise of my new approach was to make the following work:
```smt
(declare-const Coin_A Int)
(declare-const Coin_B Int)
; ...

; The first argument is the coin identifier, the second is the point in time
(declare-fun LocX (Int Int) Int)
(declare-fun LocY (Int Int) Int)
; ...

(assert
(and
    ; Constrains go here

    (exists ((T Int))
        (and
            (= (LocX Coin_A T) 9)
            (= (LocY Coin_A T) 0)
        )
    )

) ; End of AND
) ; End of ASSERT
```

---

### Progress towards solution

> `slide_puzzle_corridor.smt`

The new approach started with modeling the coins and squares, but after some thought I figured out you can model just the coins in terms of X/Y locations and constrain those, essentially mitigating the need to also model the squares in code (as seperate objects).

> `slide_puzzle_corridor_2.smt` and `slide_puzzle_corridor_3.smt`

I then decided to work on making the coins move but had a hard time gettting this to work. I eventually got this to work by creating a new smt file and stripping the issue down to its core: making a abstract function return increasing values, essentially making a single coin move to the right (possitive X)

> `slide_puzzle_corridor_4.smt`

I then reintroduced all the other stuff from the previous solution file (puzzle boundaries, extra coins, etc...) and started working on making the movement of the coin towards possitive X work given more constrains.

However, it seemed that the LocX / LocY approach was very messy (`get-model` looked very cluttered). I then spoke to Herman about visualizing the model and he gave me 2 points of feedback:
- make the puzzle smaller (less coins and less squares)
- use custom data types

> `slide_puzzle_corridor_5.smt`


## Visuallizing the output